In [1]:
# Sample Data Run
from path import Path
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import sqlalchemy as db
from sklearn import preprocessing

In [2]:
from config import password
# Set up variables to connect to SQL
database = "presidential_migration_db"
user = "postgres_blue"
password = f'{password}'
host = "presidential-migration-db.cexjpewgnljl.us-east-2.rds.amazonaws.com"
port = "5432"


In [3]:
# Create connection to Postgres DB using SQL Alchemy
engine = db.create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')
engine.connect()

In [4]:
# Pull in Election Demographics table from Postgres
df = pd.read_sql_query('SELECT * from election_dem', con=engine)
df.sort_values(by=['STATE', 'YEAR'], inplace=True)
df.set_index(['STATE', 'YEAR'], inplace=True)
df.drop(columns=['previous_senate_winning_party', 'electoral_votes'], inplace=True)
df

OUTCOME  PCT_WHITE  PCT_BLACK  PCT_HISPANIC  PCT_OVER_60  \
STATE   YEAR                                                             
ALABAMA 2000        1   0.718555   0.260610      0.017049     0.173121   
        2004        1   0.712418   0.261851      0.025003     0.178516   
        2008        1   0.706231   0.263154      0.034697     0.188939   
        2012        1   0.700032   0.264896      0.039591     0.204672   
        2016        1   0.693757   0.267632      0.042229     0.224124   
...               ...        ...        ...           ...          ...   
WYOMING 2004        1   0.945082   0.008898      0.072678     0.166543   
        2008        1   0.940193   0.008999      0.084149     0.175860   
        2012        1   0.932112   0.013311      0.093638     0.193519   
        2016        1   0.928896   0.012540      0.098770     0.218721   
        2020        1   0.925033   0.012902      0.101267     0.241318   

              PCT_UNDER_40  REGION_Midwest  REGION_Northeast  REGION_South  \
STATE   YEAR                                                                 
ALABAMA 2000      0.563600               0                 0             1   
        2004      0.543161               0                 0             1   
        2008      0.533678               0                 0             1   
        2012      0.521450               0                 0             1   
        2016      0.513533               0                 0             1   
...                    ...             ...               ...           ...   
WYOMING 2004      0.534961               0                 0             0   
        2008      0.537181               0                 0             0   
        2012      0.536336               0                 0             0   
        2016      0.535738               0                 0             0   
        2020      0.522559               0                 0             0   

              REGION_West  PAST_5_ELECTION_VOTING  bachelors_per_capita  \
STATE   YEAR                                                              
ALABAMA 2000            0                     1.0              0.064943   
        2004            0                     1.0              0.063746   
        2008            0                     1.0              0.062937   
        2012            0                     1.0              0.066716   
        2016            0                     1.0              0.068854   
...                   ...                     ...                   ...   
WYOMING 2004            1                     1.0              0.058593   
        2008            1                     1.0              0.058283   
        2012            1                     1.0              0.064167   
        2016            1                     1.0              0.074305   
        2020            1                     1.0              0.074103   

              bachelors_degree_tot  med_house_inc  violent_crime_rate  \
STATE   YEAR                                                            
ALABAMA 2000                288815          49855               21620   
        2004                288815          51395               19324   
        2008                296950          31051               21110   
        2012                321275          47091               21693   
        2016                334875          36480               25878   
...                            ...            ...                 ...   
WYOMING 2004                 29830          32023                1163   
        2008                 31825          40238                1330   
        2012                 36980          61285                1161   
        2016                 43410          47390                1431   
        2020                 44925          65134                1258   

              gdp_per_capita  guns_per_household    population  \
STATE   YEAR                                

In [5]:
# Scale data using MinMaxScaler

scaler = preprocessing.MinMaxScaler()
names = df.columns
index = df.index
d = scaler.fit_transform(df)

scaled_df = pd.DataFrame(d, columns=names, index=index)
scaled_df.reset_index(inplace=True)
scaled_df.head(20)

,STATE,YEAR,OUTCOME,PCT_WHITE,PCT_BLACK,PCT_HISPANIC,PCT_OVER_60,PCT_UNDER_40,REGION_Midwest,REGION_Northeast,...,PAST_5_ELECTION_VOTING,bachelors_per_capita,bachelors_degree_tot,med_house_inc,violent_crime_rate,gdp_per_capita,guns_per_household,population,unemployment_rate,previous_senate_outcome
0,ALABAMA,2000,1.0,0.645715,0.423755,0.021122,0.429324,0.509877,0.0,0.0,...,1.0,0.065446,0.060219,0.438072,0.100458,0.021937,0.754839,0.099456,0.233645,1.0
1,ALABAMA,2004,1.0,0.637163,0.425796,0.037494,0.455484,0.422240,0.0,0.0,...,1.0,0.060060,0.060219,0.460934,0.089525,0.051943,0.719355,0.101558,0.327103,1.0
2,ALABAMA,2008,1.0,0.628541,0.427940,0.057447,0.506023,0.381582,0.0,0.0,...,1.0,0.056422,0.062111,0.158920,0.098030,0.075129,0.698387,0.106274,0.345794,1.0
3,ALABAMA,2012,1.0,0.619903,0.430807,0.067521,0.582315,0.329151,0.0,0.0,...,1.0,0.073421,0.067767,0.397040,0.100806,0.087013,0.762903,0.108724,0.570093,1.0
4,ALABAMA,2016,1.0,0.611159,0.435310,0.072949,0.676639,0.295208,0.0,0.0,...,1.0,0.083043,0.070929,0.239515,0.120733,0.105319,0.738710,0.109930,0.355140,1.0
5,ALABAMA,2020,1.0,0.607685,0.435660,0.079760,0.748106,0.270685,0.0,0.0,...,1.0,0.093194,0.075502,0.458589,0.116772,0.122325,0.738710,0.113004,0.355140,1.0
6,ALASKA,2000,1.0,0.636703,0.055005,0.070905,0.000000,0.782547,0.0,0.0,...,1.0,0.098456,0.003601,0.454402,0.014433,0.109396,0.856452,0.003350,0.392523,1.0
7,ALASKA,2004,1.0,0.618474,0.054380,0.088679,0.076084,0.645802,0.0,0.0,...,1.0,0.082499,0.003601,0.545256,0.017314,0.166535,0.822581,0.004163,0.495327,1.0
8,ALASKA,2008,1.0,0.602351,0.052215,0.097578,0.166148,0.587660,0.0,0.0,...,1.0,0.096201,0.004538,0.168510,0.018818,0.314069,0.875806,0.004872,0.411215,1.0
9,ALASKA,2012,1.0,0.584617,0.054587,0.113286,0.271011,0.573121,0.0,0.0,...,1.0,0.115813,0.005996,0.443729,0.018518,0.310531,0.866129,0.005954,0.476636,1.0


In [6]:
# Create training set from data before 2020
training_df = scaled_df[scaled_df.YEAR < 2020]
training_df

,STATE,YEAR,OUTCOME,PCT_WHITE,PCT_BLACK,PCT_HISPANIC,PCT_OVER_60,PCT_UNDER_40,REGION_Midwest,REGION_Northeast,...,PAST_5_ELECTION_VOTING,bachelors_per_capita,bachelors_degree_tot,med_house_inc,violent_crime_rate,gdp_per_capita,guns_per_household,population,unemployment_rate,previous_senate_outcome
0,ALABAMA,2000,1.0,0.645715,0.423755,0.021122,0.429324,0.509877,0.0,0.0,...,1.0,0.065446,0.060219,0.438072,0.100458,0.021937,0.754839,0.099456,0.233645,1.0
1,ALABAMA,2004,1.0,0.637163,0.425796,0.037494,0.455484,0.422240,0.0,0.0,...,1.0,0.060060,0.060219,0.460934,0.089525,0.051943,0.719355,0.101558,0.327103,1.0
2,ALABAMA,2008,1.0,0.628541,0.427940,0.057447,0.506023,0.381582,0.0,0.0,...,1.0,0.056422,0.062111,0.158920,0.098030,0.075129,0.698387,0.106274,0.345794,1.0
3,ALABAMA,2012,1.0,0.619903,0.430807,0.067521,0.582315,0.329151,0.0,0.0,...,1.0,0.073421,0.067767,0.397040,0.100806,0.087013,0.762903,0.108724,0.570093,1.0
4,ALABAMA,2016,1.0,0.611159,0.435310,0.072949,0.676639,0.295208,0.0,0.0,...,1.0,0.083043,0.070929,0.239515,0.120733,0.105319,0.738710,0.109930,0.355140,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,WYOMING,2000,1.0,0.969169,0.008007,0.118044,0.349446,0.482344,0.0,0.0,...,1.0,0.045057,0.000000,0.422425,0.003776,0.065241,0.832258,0.000000,0.158879,1.0
301,WYOMING,2004,1.0,0.961389,0.009512,0.135625,0.397424,0.387084,0.0,0.0,...,1.0,0.036879,0.000000,0.173350,0.003048,0.125137,0.866129,0.000385,0.158879,1.0
302,WYOMING,2008,1.0,0.954577,0.009679,0.159235,0.442602,0.396602,0.0,0.0,...,1.0,0.035484,0.000464,0.295304,0.003843,0.304468,0.919355,0.001315,0.074766,1.0
303,WYOMING,2012,1.0,0.943316,0.016774,0.178766,0.528234,0.392978,0.0,0.0,...,1.0,0.061957,0.001663,0.607755,0.003038,0.242160,0.850000,0.002076,0.317757,1.0


In [7]:
# Create testing set from 2020 data
testing_df = scaled_df[scaled_df.YEAR == 2020].set_index('STATE')
testing_df

,YEAR,OUTCOME,PCT_WHITE,PCT_BLACK,PCT_HISPANIC,PCT_OVER_60,PCT_UNDER_40,REGION_Midwest,REGION_Northeast,REGION_South,...,PAST_5_ELECTION_VOTING,bachelors_per_capita,bachelors_degree_tot,med_house_inc,violent_crime_rate,gdp_per_capita,guns_per_household,population,unemployment_rate,previous_senate_outcome
STATE,,,,,,,,,,,,,,,,,,,,,
ALABAMA,2020,1.0,0.607685,0.435660,0.079760,0.748106,0.270685,0.0,0.0,1.0,...,1.0,0.093194,0.075502,0.458589,0.116772,0.122325,0.738710,0.113004,0.355140,1.0
ALASKA,2020,1.0,0.553961,0.055851,0.135750,0.496000,0.544136,0.0,0.0,0.0,...,1.0,0.144399,0.007746,0.445436,0.027713,0.236309,0.809677,0.006834,0.532710,1.0
ARIZONA,2020,0.0,0.795678,0.080107,0.639431,0.750830,0.336615,0.0,0.0,0.0,...,1.0,0.119943,0.125994,0.470777,0.155318,0.150627,0.467742,0.174221,0.542056,0.0
ARKANSAS,2020,1.0,0.745828,0.252836,0.147425,0.733672,0.321488,0.0,0.0,1.0,...,1.0,0.068816,0.040081,0.485934,0.081521,0.104736,0.722581,0.065013,0.373832,1.0
CALIFORNIA,2020,0.0,0.646881,0.101192,0.797375,0.585866,0.416241,0.0,0.0,0.0,...,0.0,0.257380,1.000000,0.465848,0.827626,0.298382,0.150000,1.000000,0.747664,0.0
COLORADO,2020,0.0,0.855916,0.070450,0.435279,0.592249,0.419774,0.0,0.0,0.0,...,0.4,0.369428,0.172428,0.518282,0.101972,0.243915,0.498387,0.134023,0.485981,0.0
CONNECTICUT,2020,0.0,0.755034,0.195468,0.332974,0.780913,0.182769,0.0,1.0,0.0,...,0.0,0.257703,0.083846,0.560176,0.028680,0.301955,0.190323,0.078793,0.542056,0.0
DELAWARE,2020,0.0,0.609040,0.376047,0.183437,0.871235,0.182751,0.0,0.0,1.0,...,0.0,0.159375,0.012854,0.520761,0.017104,0.294587,0.511290,0.012526,0.532710,0.0
DISTRICT OF COLUMBIA,2020,0.0,0.285284,0.751515,0.217824,0.418404,0.700172,0.0,0.0,1.0,...,0.0,1.000000,0.037837,0.585962,0.032761,1.000000,NaN,0.005343,0.551402,0.5


In [8]:
# Split training/testing data
y_train = training_df.OUTCOME
y_test = testing_df.OUTCOME
X_train = training_df.drop(columns=['YEAR','STATE','OUTCOME', 'guns_per_household', 'med_house_inc', 'violent_crime_rate', 'population', 'PCT_UNDER_40', 'bachelors_degree_tot'])
X_test = testing_df.drop(columns=['YEAR','OUTCOME', 'guns_per_household', 'med_house_inc', 'violent_crime_rate', 'population', 'PCT_UNDER_40', 'bachelors_degree_tot'])

In [9]:
# Find coefficients for LogisticRegression model 
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
log_odds = logreg.coef_[0]

In [10]:
pd.DataFrame(log_odds,
             X_train.columns,
             columns=['coef'])\
             .sort_values(by='coef', ascending=False)

,coef
PAST_5_ELECTION_VOTING,3.522478
REGION_South,1.068301
previous_senate_outcome,1.054014
PCT_WHITE,0.867774
PCT_OVER_60,0.626030
REGION_Midwest,0.380186
REGION_West,0.221494
PCT_BLACK,-0.441173
gdp_per_capita,-0.591360
unemployment_rate,-0.593996


In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
classifier = LogisticRegression(solver='lbfgs',
                                random_state=1)

In [12]:
classifier.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [13]:
y_pred = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test})
results

,Prediction,Actual
STATE,,
ALABAMA,1.0,1.0
ALASKA,1.0,1.0
ARIZONA,1.0,0.0
ARKANSAS,1.0,1.0
CALIFORNIA,0.0,0.0
COLORADO,0.0,0.0
CONNECTICUT,0.0,0.0
DELAWARE,0.0,0.0
DISTRICT OF COLUMBIA,0.0,0.0


In [14]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.9411764705882353


In [15]:
from sklearn.metrics import confusion_matrix, classification_report
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

[[24  2]
 [ 1 24]]


In [16]:
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

         0.0       0.96      0.92      0.94        26
         1.0       0.92      0.96      0.94        25

    accuracy                           0.94        51
   macro avg       0.94      0.94      0.94        51
weighted avg       0.94      0.94      0.94        51



In [17]:
results.to_csv('2020_predictions.csv')